In [476]:
%matplotlib inline

import requests
import shutil
import json
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Load the Twitter Trend Snapshots

Take the raw fata from dynamoDB dump, clean it up, and load it into a pandas data frame.

In [219]:
with open('output.json', 'rb') as f:
    trends = json.load(f)

In [220]:
# use this to inform cleanup process
trends["Items"][0]

{u'placeID': {u'N': u'2407517'},
 u'trend': {u'M': {u'date': {u'S': u'2016-03-03'},
   u'mappedName': {u'S': u'world book day'},
   u'placeName': {u'S': u'Fresno'},
   u'tweetName': {u'S': u'#WorldBookDay'},
   u'tweetVolume': {u'N': u'225775'},
   u'url': {u'S': u'http://twitter.com/search?q=%23WorldBookDay'}}},
 u'trendTime': {u'N': u'1457022783'}}

In [438]:
trends_clean = []

In [439]:
for trend in trends["Items"]:
    new = {}
    new["placeID"] = trend["placeID"]["N"]
    new["trendMapped"] = trend["trend"]["M"]["mappedName"]["S"]
    new["placeName"] = trend["trend"]["M"]["placeName"]["S"]
    new["trendName"] = trend["trend"]["M"]["tweetName"]["S"]
    new["trendVolume"] = trend["trend"]["M"]["tweetVolume"]["N"]
    new["searchUrl"] = trend["trend"]["M"]["url"]["S"]
    new["trendDate"] = trend["trend"]["M"]["date"]["S"]
    new["trendTime"] = trend["trendTime"]["N"]
    trends_clean.append(new)

In [440]:
# it worked
trends_clean[0]

{'placeID': u'2407517',
 'placeName': u'Fresno',
 'searchUrl': u'http://twitter.com/search?q=%23WorldBookDay',
 'trendDate': u'2016-03-03',
 'trendMapped': u'world book day',
 'trendName': u'#WorldBookDay',
 'trendTime': u'1457022783',
 'trendVolume': u'225775'}

In [444]:
df_trends = pd.DataFrame(trends_clean)

In [442]:
states = {"New York" : "NY",
"Albuquerque" : "NM",
"United States" : "XX",
"Fresno" : "CA",
"Long Beach" : "CA",
"Tucson" : "AZ",
"Philadelphia" : "PA",
"San Jose" : "CA",
"Mesa" : "AZ",
"Colorado Springs" : "CO",
"Los Angeles" : "CA",
"Washington" : "DC",
"Virginia Beach" : "VA",
"San Francisco" : "CA",
"Chicago" : "IL",
"Boston" : "MA",
"Atlanta" : "GA",
"Dallas-Ft. Worth" : "TX",
"Houston" : "TX",
"Miami" : "FL",
"Denver" : "CO",
"Cleveland" : "OH",
"Seattle" : "WA",
"Tampa" : "FL",
"Phoenix" : "AZ",
"Minneapolis" : "MN",
"San Diego" : "CA",
"Detroit" : "MI",
"Las Vegas" : "NV",
"St. Louis" : "MO",
"New Haven" : "CT",
"Charlotte" : "NC",
"Indianapolis" : "IN",
"Salt Lake City" : "UT",
"Austin" : "TX",
"Raleigh" : "NC",
"Orlando" : "FL",
"Louisville" : "KY",
"Portland" : "OR",
"Baton Rouge" : "LA",
"Columbus" : "OH",
"Pittsburgh" : "PA",
"Baltimore" : "MD",
"San Antonio" : "TX",
"Omaha" : "NE",
"Oklahoma City" : "OK",
"Jackson" : "MS",
"El Paso" : "TX",
"Milwaukee" : "WI",
"Memphis" : "TN",
"Cincinnati" : "OH",
"Providence" : "RI",
"Tallahassee" : "FL",
"Norfolk" : "VA",
"Kansas City" : "MO",
"Richmond" : "VA",
"Harrisburg" : "PA",
"Sacramento" : "CA",
"Birmingham" : "AL",
"Nashville" : "TN",
"Honolulu" : "HI",
"Greensboro" : "NC",
"Jacksonville" : "FL",
"New Orleans" : "LA"}

## Pandas Cleanup
Converting columns and shit

In [445]:
df_trends.trendTime = df_trends.trendTime.astype(int)
df_trends.trendVolume = df_trends.trendVolume.astype(int)
df_trends.index = pd.to_datetime(df_trends.trendTime, unit = 's')
df_trends = df_trends.drop("trendTime", axis = 1)
df_trends["stateName"] = df_trends.placeName.apply(states.get)
df_trends["trendMapped"] = df_trends.trendMapped.str.replace(" ", "+")

In [470]:
#group by state for full map
trends_agg = df_trends[~(df_trends["trendName"].str.contains("5SO|Leicester|ALDUB|iHeartAwards"))].groupby(["stateName","trendDate", "trendName"]).agg({"trendVolume": max})

#group by city for hover
trends_agg2= df_trends[~(df_trends["trendName"].str.contains("5SO|Leicester|ALDUB|iHeartAwards"))].groupby(["trendDate","stateName","trendName"]).max()

In [447]:
g = trends_agg['trendVolume'].groupby(level=[1,0], group_keys=False)

In [448]:
res = g.apply(lambda x: x.order(ascending=False).head(1))

/Users/michaelhi/anaconda/envs/twitter-giphy/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


In [367]:
res2 = pd.DataFrame(res).reset_index()

In [390]:
res2.index = res2.trendDate

In [83]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

print_full(trends_agg_322.index.get_level_values("placeName").value_counts())

#trends_agg_322["trendMapped"].value_counts()

New York            274
Albuquerque         273
United States       272
Fresno              272
Long Beach          271
Tucson              270
Philadelphia        267
San Jose            267
Mesa                266
Colorado Springs    265
Los Angeles         264
Washington          256
Virginia Beach      255
San Francisco       251
Chicago             242
Boston              242
Atlanta             238
Dallas-Ft. Worth    229
Houston             217
Miami               216
Denver              214
Cleveland           210
Seattle             205
Tampa               203
Phoenix             197
Minneapolis         195
San Diego           195
Detroit             190
Las Vegas           189
St. Louis           189
New Haven           189
Charlotte           188
Indianapolis        183
Salt Lake City      182
Austin              182
Raleigh             182
Orlando             177
Louisville          177
Portland            175
Baton Rouge         174
Columbus            174
Pittsburgh      

Here we get the mapped names for giphy retrieval

In [123]:
mike = [ dude.replace(" ", "+") for dude in list(set(trends_agg_322["trendMapped"]))]
responses = []

In [124]:
for trend in mike:
    resp = requests.get("http://api.giphy.com/v1/gifs/search?q=" + trend + "&api_key=dc6zaTOxFJmzC&limit=1")
    data = resp.json()
    data["trend"] = trend
    responses.append(data)
    

In [138]:
for resp in responses:
    if resp["data"]:
        url = resp["data"][0]["images"]["downsized_large"]["url"]
        res = requests.get(url, stream=True)
        with open("gifs_full/"+resp["trend"]+'.gif', 'wb') as out_file:
            shutil.copyfileobj(res.raw, out_file)
        del res

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [88]:
dude = pd.Series(["NY","NM","XX","CA","CA","AZ","PA","CA","AZ","CO",
                  "CA","DC","VA","CA","IL","MA","GA","TX","TX","FL",
                  "CO","OH","WA","FL","AZ","MN","CA","MI","NV","MO",
                  "CT","NC","IN","UT","TX","NC","FL","KY","OR","LA",
                  "OH","PA","MD","TX","NE","OK","MS","TX","WI","TN",
                  "OH","RI","FL","VA","MO","VA","PA","CA","AL","TN",
                  "HI","NC","FL","LA"])

In [471]:
trends_agg2.head()

placeID   placeName  \
trendDate  stateName trendName                            
2016-03-03 AL        Romney         2364559  Birmingham   
           AZ        #GIFparty      2471390     Phoenix   
                     #WorldBookDay  2508428      Tucson   
                     Romney         2449808        Mesa   
           CA        #GIFparty      2441472  Long Beach   

                                                                      searchUrl  \
trendDate  stateName trendName                                                    
2016-03-03 AL        Romney                  http://twitter.com/search?q=Romney   
           AZ        #GIFparty          http://twitter.com/search?q=%23GIFparty   
                     #WorldBookDay  http://twitter.com/search?q=%23WorldBookDay   
                     Romney                  http://twitter.com/search?q=Romney   
           CA        #GIFparty          http://twitter.com/search?q=%23GIFparty   

                                       trendMapped  trendVolume  
trendDate  stateName trendName                                   
2016-03-03 AL        Romney                 romney       639465  
           AZ        #GIFparty           gi+fparty       390881  
                     #WorldBookDay  world+book+day       297777  
                     Romney                 romney       526206  
           CA        #GIFparty           gi+fparty       390881

In [472]:
idx = trends_agg2.groupby(level=[0,1])["trendVolume"].transform(max) == trends_agg2["trendVolume"]

In [473]:
tops = trends_agg2[idx]

In [161]:
trends_agg_322.iloc[trends_agg_322.index.get_level_values('trendDate') == '2016-03-08'].reset_index().to_json(orient='records')

'[{"placeName":"Albuquerque","trendDate":"2016-03-08","trendName":"#DemTownHall","placeID":"2352824","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23DemTownHall","trendMapped":"dem town hall","trendVolume":77495},{"placeName":"Albuquerque","trendDate":"2016-03-08","trendName":"#InternationalWomensDay","placeID":"2352824","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23InternationalWomensDay","trendMapped":"international womens day","trendVolume":1741308},{"placeName":"Albuquerque","trendDate":"2016-03-08","trendName":"#TheBachelor","placeID":"2352824","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23TheBachelor","trendMapped":"the bachelor","trendVolume":69112},{"placeName":"Albuquerque","trendDate":"2016-03-08","trendName":"APOYO MB","placeID":"2352824","searchUrl":"http:\\/\\/twitter.com\\/search?q=%22APOYO+MB%22","trendMapped":"APOYO MB","trendVolume":135792},{"placeName":"Albuquerque","trendDate":"2016-03-08","trendName":"Roma","placeID":"2352824","searchUrl":"http:\\/\\/twitt

In [455]:
for day in list(set(list(trends_agg2.index.get_level_values("trendDate")))):
    
    df = trends_agg2.iloc[trends_agg2.index.get_level_values('trendDate') == day]
    df = df.reset_index()
    df= df.drop("trendDate",axis=1).to_json("bydate/"+day+".json",orient="records")

In [474]:
for day in list(set(list(tops.index.get_level_values("trendDate")))):
    df = tops.iloc[tops.index.get_level_values('trendDate') == day]
    df=df.reset_index()
    df.to_json("map/"+day+".json", orient="records")

In [469]:
df

,trendDate,stateName,trendName,placeID,placeName,searchUrl,trendMapped,trendVolume
0,2016-03-12,AL,#MemeHistory,2364559,Birmingham,http://twitter.com/search?q=%23MemeHistory,meme+history,897049
1,2016-03-12,AZ,#TrumpRally,2508428,Tucson,http://twitter.com/search?q=%23TrumpRally,trump+rally,532096
2,2016-03-12,CA,#MemeHistory,2488042,San Jose,http://twitter.com/search?q=%23MemeHistory,meme+history,897049
3,2016-03-12,CO,#TrumpRally,2391279,Denver,http://twitter.com/search?q=%23TrumpRally,trump+rally,635436
4,2016-03-12,CT,#TrumpRally,2458410,New Haven,http://twitter.com/search?q=%23TrumpRally,trump+rally,647745
5,2016-03-12,DC,#SLFLManila,2514815,Washington,http://twitter.com/search?q=%23SLFLManila,slfl+manila,318155
6,2016-03-12,FL,#MemeHistory,2503863,Tampa,http://twitter.com/search?q=%23MemeHistory,meme+history,900298
7,2016-03-12,GA,#MemeHistory,2357024,Atlanta,http://twitter.com/search?q=%23MemeHistory,meme+history,524782
8,2016-03-12,HI,#MemeHistory,2423945,Honolulu,http://twitter.com/search?q=%23MemeHistory,meme+history,529146
9,2016-03-12,IL,#TrumpRally,2379574,Chicago,http://twitter.com/search?q=%23TrumpRally,trump+rally,530737


In [257]:
states

{'Albuquerque': 'NM',
 'Atlanta': 'GA',
 'Baltimore': 'MD',
 'Birmingham': 'AL',
 'Boston': 'MA',
 'Chicago': 'IL',
 'Cincinnati': 'OH',
 'Denver': 'CO',
 'Detroit': 'MI',
 'El Paso': 'TX',
 'Greensboro': 'NC',
 'Harrisburg': 'PA',
 'Honolulu': 'HI',
 'Indianapolis': 'IN',
 'Jackson': 'MS',
 'Jacksonville': 'FL',
 'Kansas City': 'MO',
 'Las Vegas': 'NV',
 'Louisville': 'KY',
 'Milwaukee': 'WI',
 'Minneapolis': 'MN',
 'Nashville': 'TN',
 'New Haven': 'CT',
 'New Orleans': 'LA',
 'New York': 'NY',
 'Oklahoma City': 'OK',
 'Omaha': 'NE',
 'Phoenix': 'AZ',
 'Portland': 'OR',
 'Providence': 'RI',
 'Richmond': 'VA',
 'Sacramento': 'CA',
 'Salt Lake City': 'UT',
 'Seattle': 'WA',
 'United States': 'XX',
 'Washington': 'DC'}

In [478]:
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, 10)]

In [482]:
import os

In [486]:
[ files.rstrip(".json") for files in os.listdir("map")]

['2016-03-03',
 '2016-03-04',
 '2016-03-05',
 '2016-03-06',
 '2016-03-07',
 '2016-03-08',
 '2016-03-09',
 '2016-03-10',
 '2016-03-11',
 '2016-03-12',
 '2016-03-13',
 '2016-03-14',
 '2016-03-15',
 '2016-03-16',
 '2016-03-17',
 '2016-03-18',
 '2016-03-19',
 '2016-03-20',
 '2016-03-21',
 '2016-03-22',
 '2016-03-23',
 '2016-03-24',
 '2016-03-25',
 '2016-03-26',
 '2016-03-27',
 '2016-03-28',
 '2016-03-29',
 '2016-03-30',
 '2016-03-31',
 '2016-04-01',
 '2016-04-02',
 '2016-04-03',
 '2016-04-04',
 '2016-04-05',
 '2016-04-06',
 '2016-04-07',
 '2016-04-08',
 '2016-04-09',
 '2016-04-10',
 '2016-04-11',
 '2016-04-12',
 '2016-04-13',
 '2016-04-14',
 '2016-04-15',
 '2016-04-16',
 '2016-04-17']